In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/drive/My Drive/data/n50.csv',parse_dates=['Date'],index_col='Date')
df = df.loc["2016-01-01" : ]   #Since 2016-01-01, 5y(1238rows till 2020-12-31), + year 2021's rows
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)

In [ ]:
def number_of_years(y):        #calculates the number of years of the dataset
  p=y.index[0]                 #date of first row in the dataset (datetime format)
  q=y.index[len(y)-1]          #date of last row in the dataset  (datetime format)
  return ((q-p).days+1)/365           #the difference give the number of total days (not trading days) over the total number of years in the dataset

In [ ]:
trading_days=len(df)/number_of_years(tdf)                       #Trading days per year (automated)

In [ ]:
returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)           #calculating daily returns of the stocks in the portfolio

In [ ]:
returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
    for i in range(0,len(df)):
      if returnso[cols][i] > 0:
        returnso[cols][i]=0               #Here, returnso would mean return considered for sortino ratio

In [ ]:
covmatsh=returnsh.cov()*trading_days      #Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso=returnso.cov()*trading_days      #Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio

In [ ]:
num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
risk_free_rate = 0.0358                   #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [ ]:
portfolio_returns = []                    #initializing an empty list for portfolio returns
portfolio_volatility =[]                  #initializing an empty list for portfolio risk
stock_weights =[]                         #initializing an empty list for portfolio weights
semi_deviation =[]                        #initializing an empty list for portfolio semi-deviation
sharpe =[]                                #initializing an empty list for portfolio sharpe ratio
sortino =[]                               #initializing an empty list for portfolio sortino ratio

In [ ]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  return (a-c)/b                          #a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)

In [ ]:
for single_portfolio in range(num_portfolios):                  #iterating forloop for 50000 times to generate 50000 portfolios
  weights = np.random.random(num_assets)                        #initializing random weights
  weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
  returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
  varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
  varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
  volatility_temp = np.sqrt(varsh)                              #portfolio risk
  semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
  shtemp = ratio(returns_temp,volatility_temp,risk_free_rate)   #calculating sharpe ratio
  sotemp = ratio(returns_temp,semi_temp,risk_free_rate)         #calculating sortino ratio
  portfolio_returns.append(returns_temp)                       
  portfolio_volatility.append(volatility_temp)
  stock_weights.append(weights)
  sharpe.append(shtemp)
  sortino.append(sotemp)
  semi_deviation.append(semi_temp)

In [ ]:
portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Semi-Deviation' : semi_deviation, 'Sharpe Ratio' : sharpe, 
             'Sortino Ratio' : sortino}    
#here, 'portfolio' is a dictionary which will be used to create dataframe where each row will be a portfolio

In [ ]:
for counter,symbol in enumerate(df.columns):
  portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights] 
#to the dictionary (named 'portfolio'), weights for each symbol are added in so as to be displayed in the dataframe

In [ ]:
pc = pd.DataFrame(portfolio)         #making the final dataframe where data of 50000 portfolios is appended (subject to be saved, whose code is to be written)

In [ ]:
pc=pc*100                                       #Converting everything to percentage
pc['Sharpe Ratio']=pc['Sharpe Ratio']/100       #leaving ratios as it is
pc['Sortino Ratio']=pc['Sortino Ratio']/100

In [ ]:
pc.to_csv('/content/drive/My Drive/experiment/take 14-03-2021/portfolios_by_MV.csv')  #saving the portfolios data

In [ ]:
max_sharpe=pc['Sharpe Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
max_sharpe_portfolio=pc.loc[pc['Sharpe Ratio'] == max_sharpe]
max_sharpe_portfolio

,Returns,Standard Deviation,Semi-Deviation,Sharpe Ratio,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,HCLTECH Weight,HDFCBANK Weight,HDFCLIFE Weight,HINDALCO Weight,HINDUNILVR Weight,HDFC Weight,ICICIBANK Weight,INFY Weight,JSWSTEEL Weight,KOTAKBANK Weight,NESTLEIND Weight,RELIANCE Weight,SHREECEM Weight,TCS Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,WIPRO Weight
9182,28.988566,17.11367,10.690721,1.484694,2.376693,1.549459,8.697908,7.309154,7.867606,8.534857,6.696085,0.079932,6.84089,0.402327,1.315497,0.313284,0.305251,3.464734,0.302845,5.582477,9.844509,7.618349,0.224515,2.688051,5.75369,0.867298,7.651331,6.08995


In [ ]:
max_sortino=pc['Sortino Ratio'].max()                                            #Best optimised portfolio wrt sortino ratio
max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
max_sortino_portfolio

,Returns,Standard Deviation,Semi-Deviation,Sharpe Ratio,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,HCLTECH Weight,HDFCBANK Weight,HDFCLIFE Weight,HINDALCO Weight,HINDUNILVR Weight,HDFC Weight,ICICIBANK Weight,INFY Weight,JSWSTEEL Weight,KOTAKBANK Weight,NESTLEIND Weight,RELIANCE Weight,SHREECEM Weight,TCS Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,WIPRO Weight
37580,26.885501,15.734624,9.770734,1.48116,2.385235,9.173832,2.714857,2.375925,4.012207,6.685499,7.913618,0.618354,8.345997,0.775771,5.653637,0.868243,1.878566,3.245976,3.387397,0.649569,9.309904,8.020694,1.515263,9.350998,2.997622,3.98125,6.109733,0.415089


In [ ]:
#code for visualization is to be written